In [ ]:
import numpy as np
import PIL
import tensorflow as tf

from tensorflow import keras

In [ ]:
import pathlib
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = keras.utils.get_file('flower_photos', origin=dataset_url, untar=True)
data_dir = pathlib.Path(data_dir)
data_dir

228813984/228813984 [==============================] - 1s 0us/step


PosixPath('/root/.keras/datasets/flower_photos')

In [ ]:
image_count = len(list(data_dir.glob('*/*.jpg')))
print(image_count)

3670


In [ ]:
batch_size = 32
img_height = 180
img_width = 180

ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size
)

train_ds, test_ds = tf.keras.utils.split_dataset(
    ds,
    left_size=0.8,
    shuffle=True,
    seed=42
)


Found 3670 files belonging to 5 classes.


In [ ]:
num_classes = 5

inputs = keras.Input(shape=(img_height, img_width, 3))
x = keras.layers.Rescaling(1./255)(inputs)
x = keras.layers.RandomRotation(factor=0.2)(x)
x = keras.layers.RandomZoom(0.2)(x)
x = keras.layers.RandomContrast(0.2)(x)
x = keras.layers.Conv2D(16, 3, padding="same", activation="relu")(x)
x = keras.layers.MaxPooling2D()(x)
x = keras.layers.Conv2D(32, 3, padding='same', activation='relu')(x)
x = keras.layers.Dropout(0.2)(x)
x = keras.layers.MaxPooling2D()(x)
x = keras.layers.Conv2D(64, 3, padding='same', activation='relu')(x)
x = keras.layers.Dropout(0.2)(x)
x = keras.layers.MaxPooling2D()(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(128, activation='relu')(x)
x = keras.layers.Dropout(0.2)(x)
x = keras.layers.Dense(128, activation='relu')(x)
outputs = keras.layers.Dense(num_classes)(x)

model = keras.Model(inputs=inputs, outputs=outputs)
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 180, 180, 3)]     0         
                                                                 
 rescaling_2 (Rescaling)     (None, 180, 180, 3)       0         
                                                                 
 random_rotation_2 (RandomRo  (None, 180, 180, 3)      0         
 tation)                                                         
                                                                 
 random_zoom_2 (RandomZoom)  (None, 180, 180, 3)       0         
                                                                 
 random_contrast_2 (RandomCo  (None, 180, 180, 3)      0         
 ntrast)                                                         
                                                                 
 conv2d_6 (Conv2D)           (None, 180, 180, 16)      448 

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)

In [ ]:
model.compile(optimizer="adam", loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
epochs=30
history = model.fit(
  train_ds,
  epochs=epochs
)

Epoch 1/30
92/92 [==============================] - 6s 30ms/step - loss: 1.4031 - accuracy: 0.3798
Epoch 2/30
92/92 [==============================] - 3s 28ms/step - loss: 1.1454 - accuracy: 0.5245
Epoch 3/30
92/92 [==============================] - 3s 28ms/step - loss: 1.0892 - accuracy: 0.5545
Epoch 4/30
92/92 [==============================] - 3s 29ms/step - loss: 1.0103 - accuracy: 0.5916
Epoch 5/30
92/92 [==============================] - 3s 29ms/step - loss: 0.9426 - accuracy: 0.6257
Epoch 6/30
92/92 [==============================] - 3s 29ms/step - loss: 0.8944 - accuracy: 0.6543
Epoch 7/30
92/92 [==============================] - 3s 29ms/step - loss: 0.8578 - accuracy: 0.6696
Epoch 8/30
92/92 [==============================] - 3s 28ms/step - loss: 0.8231 - accuracy: 0.6764
Epoch 9/30
92/92 [==============================] - 3s 29ms/step - loss: 0.8240 - accuracy: 0.6788
Epoch 10/30
92/92 [==============================] - 3s 28ms/step - loss: 0.7771 - accuracy: 0.7006
Epoch 11/

In [ ]:
model.evaluate(test_ds)

23/23 [==============================] - 0s 12ms/step - loss: 0.6789 - accuracy: 0.7411


[0.6788836717605591, 0.7411444187164307]